In [81]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame

import plotly.express as px


import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

import time

2.6.0+cu126
12.6
90501
CUDA available: True
GPU: NVIDIA GeForce RTX 4070 SUPER


In [82]:
import os
from dotenv import load_dotenv
from pathlib import Path

# Path to the .env file inside 01 - Documentation
env_path = Path("01 - Documentation") / "keys.env"

# Load the .env file from that path
load_dotenv(dotenv_path=env_path)

# Now you can access your keys
API_KEY = os.getenv("ALPACA_API_KEY")
SECRET_KEY = os.getenv("ALPACA_SECRET_KEY")

print(API_KEY[:4] + "****")  # optional: verify it's working

PKD5****


In [83]:
myAssets = ["AMZN", "META", "GOOGL", "AMD", "NKE", "UBER", "COST", "JPM", "CRM", "TXRH"]
shareCount = [15, 4.53, 14.43, 25.21, 30.16, 20, 1.25, 4.84, 4.36, 5.68]

client = StockHistoricalDataClient(API_KEY, SECRET_KEY)

end_date = datetime.today()
start_date = end_date - timedelta(days=365)  # 1 year of daily prices

request = StockBarsRequest(
    symbol_or_symbols=myAssets,
    timeframe=TimeFrame.Day,
    start=start_date,
    end=end_date
)

bars = client.get_stock_bars(request).df

# Get the last row (most recent date) per asset
latest_closes = bars['close'].groupby(level=0).last()
prices = latest_closes.to_dict()

print(prices)


print(bars.head())
print(bars.columns)


{'AMD': 88.29, 'AMZN': 174.33, 'COST': 967.75, 'CRM': 249.84, 'GOOGL': 153.33, 'JPM': 229.61, 'META': 502.31, 'NKE': 53.55, 'TXRH': 160.51, 'UBER': 73.06}
                                    open     high     low   close      volume  \
symbol timestamp                                                                
AMD    2024-04-17 04:00:00+00:00  163.97  164.450  153.88  154.02  75908969.0   
       2024-04-18 04:00:00+00:00  155.51  156.960  152.32  155.08  52669816.0   
       2024-04-19 04:00:00+00:00  151.59  154.250  145.29  146.64  71618197.0   
       2024-04-22 04:00:00+00:00  148.15  149.890  145.63  148.64  49859635.0   
       2024-04-23 04:00:00+00:00  151.65  153.495  150.35  152.27  46051911.0   

                                  trade_count        vwap  
symbol timestamp                                           
AMD    2024-04-17 04:00:00+00:00     646783.0  157.696926  
       2024-04-18 04:00:00+00:00     513832.0  155.017218  
       2024-04-19 04:00:00+00:00     

In [84]:
# Calculate the value of each position
position_values = [shareCount[i] * prices[symbol] for i, symbol in enumerate(myAssets)]

# Create a DataFrame for visualization
Assetsdf = pd.DataFrame({
    "Symbol": myAssets,
    "Shares": shareCount,
    "Price": [prices[symbol] for symbol in myAssets],
    "Value": position_values
})

Assetsdf

,Symbol,Shares,Price,Value
0,AMZN,15.00,174.33,2614.9500
1,META,4.53,502.31,2275.4643
2,GOOGL,14.43,153.33,2212.5519
3,AMD,25.21,88.29,2225.7909
4,NKE,30.16,53.55,1615.0680
5,UBER,20.00,73.06,1461.2000
6,COST,1.25,967.75,1209.6875
7,JPM,4.84,229.61,1111.3124
8,CRM,4.36,249.84,1089.3024
9,TXRH,5.68,160.51,911.6968


In [85]:
portfolioValue = sum(Assetsdf["Value"])
print("Portfolio Value: $", portfolioValue)

fig = px.pie(Assetsdf,
             names='Symbol',
             values='Value',
             title='Portfolio Allocation by Market Value: $' + str(portfolioValue),
             hole=0,  # Set to >0 if you want a donut chart instead
             width=800,
             height=500)

# Add ticker symbols on each slice
fig.update_traces(textinfo='label+percent', textposition='inside')

fig.show()


Portfolio Value: $ 16727.0242


In [86]:
import plotly.graph_objects as go

close_prices = bars.reset_index().pivot(index="timestamp", columns="symbol", values="close").sort_index()

normalized_prices = close_prices / close_prices.iloc[0]

fig = go.Figure()

for symbol in myAssets:
    fig.add_trace(go.Scatter(
        x=normalized_prices.index,
        y=normalized_prices[symbol],
        mode='lines',
        name=symbol
    ))

fig.update_layout(
    title='Normalized Portfolio Stock Prices (Base = 1)',
    xaxis_title='Date',
    yaxis_title='Normalized Price',
    width=800,
    height=600
)

fig.show()


### Sharpe Ratio

The **Sharpe Ratio** measures the risk-adjusted return of a portfolio or investment. It indicates how much **excess return** you earn for each unit of **volatility** (risk).

It is defined as:

$$
\text{Sharpe Ratio} = \frac{R_p - R_f}{\sigma_p}
$$

Where:

- $R_p$ = Portfolio return (mean)  
- $R_f$ = Risk-free rate of return  
- $\sigma_p$ = Standard deviation of the portfolio returns  

A higher Sharpe ratio indicates a more favorable risk-adjusted performance.


In [90]:
# Pivot to get multi-asset wide format (Date x Symbols)
returns = close_prices.pct_change().dropna()

# Get normalized weights
initial_prices = close_prices.iloc[0]
weights = pd.Series(shareCount, index=myAssets)
weights = weights / (weights * initial_prices).sum()

# Portfolio daily returns
portfolio_returns = returns.dot(weights)

risk_free_rate = 0.043070  # 4% annual
trading_days = 252

# Convert Rf to daily rate
daily_rf = risk_free_rate / trading_days

excess_returns = portfolio_returns - daily_rf

sharpe_ratio = (excess_returns.mean() / excess_returns.std()) * np.sqrt(trading_days)
print(f"📊 Sharpe Ratio: {sharpe_ratio:.2f}")
print("Portfolio mean daily return:", portfolio_returns.mean())
print("Portfolio daily std dev:", portfolio_returns.std())
print("Risk-free daily:", daily_rf)
print("Excess return mean:", excess_returns.mean())
print("Sharpe Ratio:", sharpe_ratio)



📊 Sharpe Ratio: -24.22
Portfolio mean daily return: -4.114472539365586e-06
Portfolio daily std dev: 0.00011470858753973332
Risk-free daily: 0.0001709126984126984
Excess return mean: -0.000175027170952064
Sharpe Ratio: -24.221989492701905


In [88]:
cumulative_returns = (1 + portfolio_returns).cumprod()

fig = go.Figure()
fig.add_trace(go.Scatter(x=cumulative_returns.index, y=cumulative_returns, name='Portfolio Value'))
fig.update_layout(title='Cumulative Portfolio Value (Normalized)',
                  xaxis_title='Date', yaxis_title='Growth Factor')
fig.show()
